<a href="https://colab.research.google.com/github/rkmad/genAI_beer_recipes/blob/main/beer_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Investigate sources of beer recipes
- Scrape beer recipes from brewersfriend.com and punkapi.com
- Normalize hops names according to hops characteristic table from homebrew academy
- Clean and normalize recipes


In [ ]:
pip install flatten_json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7980 sha256=5b2a08cf2add79c4fd2a660c092d1fc2ea20657390239a62ab1f0db6ab43652e
  Stored in directory: /root/.cache/pip/wheels/e6/b3/2a/beb2ceb72d11bf335f9c2f87aae26981f6744f3fc885cde665
Successfully built flatten-json


In [ ]:
import requests
import json
from flatten_json import flatten
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
# curl -H "X-API-Key: 535ad06757dd29bd83be1e635ec10530407a8XAVTX" \
# https://api.brewersfriend.com/v1/recipes/573389983.xml

# # convert curl to python
# # https://curlconverter.com/

In [ ]:
# headers = {
#     'X-API-Key': '535ad06757dd29bd83be1e635ec10530407a8XAVTX',
#     'User-Agent': 'Chrome'
# }

# response = requests.get('https://api.brewersfriend.com/v1/recipes/573389983.xml', headers=headers)

In [ ]:
# response.json()

In [ ]:
"""
API documentation
https://punkapi.com/documentation/v2
"""

response = requests.get('https://api.punkapi.com/v2/beers?page=5&per_page=80')

# 325 total beers
recipes= response.json()

In [ ]:
json_out= []
for num in range(1,6):
  url_txt = f"https://api.punkapi.com/v2/beers?page={num}&per_page=80"
  response = requests.get(url_txt)
  json_out.append(response.json())
# flatten list into single list of dicts
recipe_list = [item for sublist in json_out for item in sublist]

In [ ]:
recipe_list[324]

{'id': 325,
 'name': 'Zipcode',
 'tagline': 'Pilsner.',
 'first_brewed': '2018',
 'description': 'A new 5% Pilsner, that feaures in Fanzine, our new subscription model.',
 'image_url': None,
 'abv': 5,
 'ibu': 37,
 'target_fg': 1006,
 'target_og': 1044,
 'ebc': 5,
 'srm': 3,
 'ph': 4.4,
 'attenuation_level': 86,
 'volume': {'value': 20, 'unit': 'litres'},
 'boil_volume': {'value': 25, 'unit': 'litres'},
 'method': {'mash_temp': [{'temp': {'value': 50, 'unit': 'celsius'},
    'duration': 50}],
  'fermentation': {'temp': {'value': 11, 'unit': 'celsius'}},
  'twist': None},
 'ingredients': {'malt': [{'name': 'Pilsner',
    'amount': {'value': 3.54, 'unit': 'kilograms'}},
   {'name': 'Carapils', 'amount': {'value': 0.24, 'unit': 'kilograms'}}],
  'hops': [{'name': 'Magnum',
    'amount': {'value': 12, 'unit': 'grams'},
    'add': '70',
    'attribute': 'Bittering'},
   {'name': 'Spalter',
    'amount': {'value': 15, 'unit': 'grams'},
    'add': '30',
    'attribute': 'Flavour'},
   {'name'

In [ ]:
#flatten(recipe_list[324])
recipe_list_flat = [flatten(d) for d in recipe_list]

In [ ]:
recipe_list_flat

In [ ]:
df =  pd.json_normalize(
    recipe_list,
    #record_path=['ingredients'],
    meta=["method", "ingredients"],

    errors = 'ignore'
)

In [ ]:
df = pd.DataFrame(recipe_list_flat)
df

,id,name,tagline,first_brewed,description,image_url,abv,ibu,target_fg,target_og,...,ingredients_hops_16_name,ingredients_hops_16_amount_value,ingredients_hops_16_amount_unit,ingredients_hops_16_add,ingredients_hops_16_attribute,ingredients_hops_17_name,ingredients_hops_17_amount_value,ingredients_hops_17_amount_unit,ingredients_hops_17_add,ingredients_hops_17_attribute
0,1,Buzz,A Real Bitter Experience.,09/2007,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.5,60.0,1010.0,1044.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Trashy Blonde,You Know You Shouldn't,04/2008,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.1,41.5,1010.0,1041.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Berliner Weisse With Yuzu - B-Sides,Japanese Citrus Berliner Weisse.,11/2015,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,4.2,8.0,1007.0,1040.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Pilsen Lager,Unleash the Yeast Series.,09/2013,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,6.3,55.0,1012.0,1060.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Avery Brown Dredge,Bloggers' Imperial Pilsner.,02/2011,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,7.2,59.0,1027.0,1069.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,321,Jinx Pale Ale,American Pale Ale.,2018,"A new 4.7% ABV American pale ale, Jinx Pale Al...",None,4.7,35.0,1008.0,1043.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,322,Jet Trash,West Coast IPA.,2018,"A new 6.9% ABV West Coast IPA, Jet Trash is on...",None,6.9,65.0,1008.0,1060.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,323,Interstate Vienna Lager.,Vienna Lager.,2018,"A new 5% ABV Vienna Lager, that features in Fa...",None,5.0,25.0,1007.0,1045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,324,Fools Gold Dortmunder Lager,Lager.,2018,"A new 5.6% Dortmunder Lager, that featues in F...",None,5.6,32.0,1009.0,1052.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df

In [ ]:
# save recipe dataframe
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
df.to_csv("/content/drive/MyDrive/DS_projects/ProjectsWithAle/beer_data/brewdog_data_flat.csv")

Mounted at /content/drive


# NORMALIZE BREWDOG RECIPE HOP NAMES
LOAD CSV RECIPES HERE

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DS_projects/ProjectsWithAle/beer_data/brewdog_data_flat.csv")

Import hops characteristic table from homebrew academy and clean and normalize hops names

In [ ]:
# import hops characteristic table from homebrew academy
import html
url = 'https://homebrewacademy.com/hops-chart/'
# get uncompressed response
headers = {'Accept-Encoding': 'identity', 'User-Agent': 'Custom user agent'}
r = requests.get(url, headers=headers)
r.headers
r.text

'<!DOCTYPE html>\n<html lang="en-US" prefix="og: http://ogp.me/ns#">\n<head><meta charset="UTF-8" /><script>if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\\/7\\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}</script><script>class RocketLazyLoadScripts{constructor(){this.triggerEvents=["keydown","mousedown","mousemove","touchmove","touchstart","touchend","wheel"],this.userEventHandler=this._triggerListener.bind(this),this.touchStartHandler=this._onTouchStart.bind(this),this.touchMoveHandler=this._onTouchMove.bind(this),this.touchEndHandler=this._onTouchEnd.bind(this),this.clickHandler=this._onClick.bind(this),this.

In [ ]:
# No need to use beautiful soup to get table in a url,
# can do with Pandas, see below...
# from bs4 import BeautifulSoup
# html=  r.text
# soup = BeautifulSoup(html, "html.parser")
# #print(soup.body.get_text().strip())


In [ ]:
# get hops table data as dataframe using Pandas
tables = pd.read_html(url)

In [ ]:
df_hops= tables[0]

In [ ]:
df_hops

,Name,Alpha,Beta,Origin,Description,Aroma,Typical Beer Styles,Used For,Substitutions
0,Ahtanum™,5.7-6.3%,5.0-6.5%,American,"Floral, earthy, citrus and grapefruit tones","Citrus,Floral,Earthy","American Amber,American Pale Ale,Brown Ale,Ind...",Aroma,"Amarillo®, Cascade"
1,Amarillo®,8.0-11.0%,6.0-7.0%,American,"Floral, tropical, and citrus (lemon, orange an...","Citrus,Floral,Tropical Fruit","American Amber,American Pale Ale,India Pale Al...",Aroma,"Cascade, Centennial"
2,Apollo,18.0-21.0%,%,American,NaN,NaN,"American Amber,American Pale Ale,Barleywine,Im...",Bittering,Zeus
3,Aquila,6.0-7.5%,%,American,NaN,NaN,NaN,Aroma,NaN
4,Australian Ella,14.0-16.0%,4.0-7.5%,Australian,"Spicy, hoppy and floral with hints of anise","Floral,Spicy,Herbal","American Lager,European Pale Lager",Aroma,NaN
...,...,...,...,...,...,...,...,...,...
145,Northwest Golding,4.0-5.0%,%,NaN,Known for aromatic qaulities.,NaN,"Brown Ale,English Strong Ale,Scottish Strong A...",Aroma,NaN
146,UK Kent Golding,4.0-5.5%,%,British,"Gentle, fragrant and pleasant.",NaN,"English Strong Ale,Scottish Strong Ale,English...",Aroma,"UK Progress, Golding, UK WGV"
147,Argentina Cascade,3.0-5.0%,%,Argentina,Similar to US Cascade hops but with more of a ...,"Citrus,Spicy","English Bitter,English Pale Ale,European Dark ...",NaN,Any noble hops
148,Polish Marynka,9.0-12.0%,%,Polish,New hop variety with intense armoa,NaN,"European Dark Lager,European Pale Lager",Aroma,NaN


In [ ]:
# TO DO:
# DONE refactor hops names in substitutions column into list
# DONE remove special characters from hops names in columns: "Name", "Substitutions"
# df_hops df, "Substitutions" if contains "noble", expand to noble_hops values
# DONE map dict of norm names to columns of brewdog

def namecleaner (name: str):
  """
  String method to clean hop names
  - removes special characters: TM sign, copyright sign, registered sign
  """
  pass


# remove special characters from hops names,
df_hops["Name"] = df_hops["Name"].str.replace(u"\u2122", '') # TM sign
#df_hops["Name"].str.replace(u"\u00A9", '') # copyright sign
df_hops["Name"] = df_hops["Name"].str.replace(u"\u00AE", '') #registered sign

df_hops["Substitutions"] = df_hops["Substitutions"].str.replace(u"\u2122", '') # TM sign
df_hops["Substitutions"] = df_hops["Substitutions"].str.replace(u"\u00AE", '') #registered sign
df_hops["Substitutions"] = df_hops["Substitutions"].str.split(", ") # split into list

# normalize hop names to be compatible with recipes:
# keys: names used in homebrew academy hops table
# values: names used in brewdog recipes
norm_names = {
    "UK First Gold": "First Gold",
    "UK Challenger": "Challenger",
    "UK Bramling Cross": "Bramling Cross",
    "Golding": "Goldings",
    "German Tettnanger": "Tettnang",
    "STYRIAN GOLDING": "STYRIAN GOLDINGS",
    "Styrian Bobek": "Bobek",
    "New Zealand Waimea": "Waimea",
    "New Zealand Pacifica":"Pacifica",
    "New Zealand Pacific Jade": "Pacific Jade",
    "New Zealand Pacific Gem": "Pacific Gem",
    "New Zealand Nelson Sauvin":"Nelson Sauvin",
    "New Zealand Motueka": "Motueka",
    "New Zealand Kohatu":"Kohatu",
    "New Zealand Green Bullet":"Green Bullet",
    "German Spalt": "Spalter",
    "German Saphir":"Sapphire",
    "German Mandarina Bavaria" : "Mandarina Bavaria",
    "German Hersbruker":"Hersbrucker",
    "German Herkules":"Hercules",
    "German Hallertau Mittelfrüh":"Hallertau Mittelfrüh",
    "German Hallertau Blanc":"Hallertau Blanc",
    "UK Fuggle":"Fuggles",
    "Australian Vic Secret":"Vic Secret",
    "Australian Galaxy": "Galaxy",
    "Australian Ella": "Ella",
    "Czech Saaz":"Saaz"
  }
noble_hops = {
  "German Hallertau Mittelfrüh":"Hallertau Mittelfrüh",
  "German Spalt": "Spalter",
  "German Tettnanger": "Tettnang",
  "Czech Saaz":"Saaz"
  }

# normalize hop names to be compatible with recipes, all upper case:
# invert dictionaries to be more broadly applicable with other recipe sources
# keys (OLD): names used in brewdog recipes
# values (NEW): names used in homebrew academy hops table
norm_names = { v.upper():k.upper() for k, v in norm_names.items()}
noble_hops = { v.upper():k.upper() for k, v in noble_hops.items()}

In [ ]:
list(noble_hops.values())

['GERMAN HALLERTAU MITTELFRÜH',
 'GERMAN SPALT',
 'GERMAN TETTNANGER',
 'CZECH SAAZ']

In [ ]:
norm_names = { v.upper():k.upper() for k, v in norm_names.items()}
noble_hops = { v.upper():k.upper() for k, v in noble_hops.items()}

In [ ]:
# get list of unique hop names from brewdog recipes
hops_list=set()
for num in range(0,18):
  col_name= f'ingredients_hops_{num}_name'
  currhops = df[col_name][df[col_name].notna()].str.upper().unique()
  hops_list.update(set(currhops))


In [ ]:
# normalize brewdog hops names
# check for "noble hops"
# List of indices with "noble hops"
noble_index = []
for num in range(0,18):
  col_name= f'ingredients_hops_{num}_name'
  currhops = df[col_name][df[col_name].notna()].str.contains("noble")
  noble_index.append(currhops)
df_noble = df[df.apply(lambda row: row.astype(str).str.contains('noble').any(), axis=1)== True]
col_list=[]
for col in df_noble.columns:
  row = df_noble[col].astype(str).str.contains('noble').any()
  col_list.append(row)

# only one instance of the word noble in entire df of brewdog recipes
col_list.count(True)
df_noble.iloc[0,5]

In [ ]:
# normalize brewdog hops names to homebrew academy hops table
for num in range(0,18):
  col_name= f'ingredients_hops_{num}_name'
  df[col_name] = df[col_name].astype(str).str.upper()
  df[col_name].replace(to_replace= norm_names, inplace=True)

In [ ]:
# get list of unique hop names from hops table
# get list of unique hop names from brewdog recipes
hops_list=set()
for num in range(0,18):
  col_name= f'ingredients_hops_{num}_name'
  currhops = df[col_name][df[col_name].notna()].str.upper().unique()
  hops_list.update(set(currhops))

df_hops["Name"] = df_hops["Name"].str.upper()
df_hops["Name"][df_hops["Name"].isin(list(hops_list))]

df_hops["Name"][df_hops["Name"].isin(list(hops_list))]
# 48 hops in hops table are in brew dog recipes

0                        AHTANUM
1                       AMARILLO
2                         APOLLO
4                AUSTRALIAN ELLA
5              AUSTRALIAN GALAXY
12         AUSTRALIAN VIC SECRET
16                         BRAVO
20                       CASCADE
22                    CENTENNIAL
24                       CHINOOK
25                         CITRA
28                      COLUMBUS
29                         COMET
30                       CRYSTAL
32                    CZECH SAAZ
34                     EL DORADO
35                       EQUINOX
45                        GALENA
51               GERMAN HERKULES
52             GERMAN HERSBRUKER
55      GERMAN MANDARINA BAVARIA
63                  GERMAN SPALT
65             GERMAN TETTNANGER
68                       GOLDING
72                       LIBERTY
73                        MAGNUM
75                        MOSAIC
80      NEW ZEALAND GREEN BULLET
81            NEW ZEALAND KOHATU
82           NEW ZEALAND MOTUEKA
83     NEW

In [ ]:
df.to_csv("/content/drive/MyDrive/DS_projects/ProjectsWithAle/beer_data/brewdog_data_normalized.csv")

# Brewdog recipes hops normalized

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DS_projects/ProjectsWithAle/beer_data/brewdog_data_normalized.csv")

In [ ]:
df

,Unnamed: 0,Unnamed: 0.1,id,name,tagline,first_brewed,description,image_url,abv,ibu,...,ingredients_hops_16_name,ingredients_hops_16_amount_value,ingredients_hops_16_amount_unit,ingredients_hops_16_add,ingredients_hops_16_attribute,ingredients_hops_17_name,ingredients_hops_17_amount_value,ingredients_hops_17_amount_unit,ingredients_hops_17_add,ingredients_hops_17_attribute
0,0,0,1,Buzz,A Real Bitter Experience.,09/2007,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.5,60.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
1,1,1,2,Trashy Blonde,You Know You Shouldn't,04/2008,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.1,41.5,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
2,2,2,3,Berliner Weisse With Yuzu - B-Sides,Japanese Citrus Berliner Weisse.,11/2015,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,4.2,8.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
3,3,3,4,Pilsen Lager,Unleash the Yeast Series.,09/2013,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,6.3,55.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
4,4,4,5,Avery Brown Dredge,Bloggers' Imperial Pilsner.,02/2011,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,7.2,59.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,320,320,321,Jinx Pale Ale,American Pale Ale.,2018,"A new 4.7% ABV American pale ale, Jinx Pale Al...",NaN,4.7,35.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
321,321,321,322,Jet Trash,West Coast IPA.,2018,"A new 6.9% ABV West Coast IPA, Jet Trash is on...",NaN,6.9,65.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
322,322,322,323,Interstate Vienna Lager.,Vienna Lager.,2018,"A new 5% ABV Vienna Lager, that features in Fa...",NaN,5.0,25.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN
323,323,323,324,Fools Gold Dortmunder Lager,Lager.,2018,"A new 5.6% Dortmunder Lager, that featues in F...",NaN,5.6,32.0,...,NAN,NaN,NaN,NaN,NaN,NAN,NaN,NaN,NaN,NaN


# Check out brewersfriend beer recipes
https://www.brewersfriend.com/homebrew-recipes/

https://www.brewersfriend.com/homebrew-recipes/page/2/

https://www.brewersfriend.com/homebrew-recipes/page/13073/

In [ ]:
import requests

url = "http://www.brewersfriend.com/homebrew-recipes/"

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text, flavor="bs4")
# generates problem with table parsing, data is there but is separated into ~40 separate tables

# probably better to use beautiful soup
#https://stackoverflow.com/questions/70179476/separate-table-row-to-2-when-scraping-with-pandas-read-html
#https://stackoverflow.com/questions/41447277/trouble-using-pandas-read-html


In [ ]:
len(dfs)

36

In [ ]:
# try to scrape Beersmith tables


url = "https://beersmithrecipes.com/toprated/3"

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text, flavor="bs4")

ValueError: ignored